# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

As a part of Generative Artifcial Intelligence Nanodegree program by Udacity, this is my first project on "Apply Lightweight Fine-Tuning to a Foundation Model" Lightweight fine-tuning is one of the most important techniques for adapting foundation models, because it allows is to modify foundation models for 
our needs without needing substantial computational resources.

Parameter-efficient fine-tuning is a technique to use pre-trained machine learning models with minimal additional data or computational resources. Basically in this technique, we use specific parameters rather than retraining the entire model. This approach, often used in transfer learning, accelerates convergence and enhances generalization, particularly in NLP, computer vision, and related fields.

In this project, I've applied parameter-efficient fine-tuning using the Hugging Face peft library. In order to do, I've followed following steps.

Step 1: Load a Pre-trained Model and Evaluate Its Performance
I have used thee "distilbert-base-uncased" model from Hugging Face peft library. I  have used the subset of Amazon Polarity dataset. As entire dataset was quite heavy.
In this step, I have prepared the dataset and have evaluated the pre-trained model's performance on the validation dataset.

Step 2: Perform Parameter-Efficient Fine-Tuning Using the Pre-trained Model
In this step, I've defined training arguments for fine-tuning and have also Fine-tuned the pre-trained model on the training subset.
I've also saved the fine-tuned model.

Step 3: Perform Inference Using the Fine-Tuned Model and Compare Its Performance to the Original Model
In this step, I've loaded the fine-tuned model and have evaluated the fine-tuned model's performance on the validation dataset.
I've also compared the trainer evaluation results with the final fine-tuned evaluation results.

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
#pip install scikit-learn torch transformers datasets 

In [2]:
# This step is to import libaries
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset, load_metric

# This step is to create tokenizer from pre-trained model "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# This step is to load Amazon Polarity dataset and to create a small subset for training and validation
ds_amazon_polarity = load_dataset("amazon_polarity")
train_dataset = ds_amazon_polarity["train"].shuffle(seed=42).select(range(2000))
val_dataset = ds_amazon_polarity["test"].shuffle(seed=42).select(range(400))

# This step is to Tokenize datasets
def preprocess_function(examples):
    return tokenizer(examples["content"], truncation=True, padding="max_length", max_length=128)

tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)

metric = load_metric("accuracy", trust_remote_code=True)

def compute_metrics(eval_op):
    predictions = eval_op.predictions.argmax(-1)
    return metric.compute(predictions=predictions, references=eval_op.label_ids)

# This step is to set training arguments and to evaluate Trainer
training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=30,
    logging_dir="./logs",
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics,
)

# Evaluate pre-trained model performance
Evaluation_result = trainer.evaluate()
print(f"Evaluation results on Trainer: {Evaluation_result}")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_553/4284527552.py:21: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy", trust_remote_code=True)


Evaluation results on Trainer: {'eval_loss': 0.6916698217391968, 'eval_accuracy': 0.5125, 'eval_runtime': 2.1527, 'eval_samples_per_second': 185.814, 'eval_steps_per_second': 6.504}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [3]:
# These steps are for training arguments for fine-tuning
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1, 
    per_device_train_batch_size=30,
    per_device_eval_batch_size=30,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics,
)

# Fine-tune the model
trainer.train()
trainer.save_model("./peft_model")

Step,Training Loss
10,0.689200
20,0.690100
30,0.689700
40,0.685000
50,0.678000
60,0.661900


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [4]:
# This step is to load the fine-tuned model and to use trainer for evaluation
f_tuned_model = AutoModelForSequenceClassification.from_pretrained("./peft_model")

trainer = Trainer(
    model=f_tuned_model,
    args=training_args,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics,
)

Evaluation_result_f = trainer.evaluate()
print(f"Final evaluation result: {Evaluation_result_f}")

# This step is to show results
print(f"Please see below to get compariosn details of results:")
print(f"First evaluation accuracy results: {Evaluation_result['eval_accuracy']}")
print(f"Final evaluation accuray results: {Evaluation_result_f['eval_accuracy']}")

Final evaluation result: {'eval_loss': 0.6143025755882263, 'eval_accuracy': 0.8425, 'eval_runtime': 1.4562, 'eval_samples_per_second': 274.678, 'eval_steps_per_second': 9.614}
Please see below to get compariosn details of results:
First evaluation accuracy results: 0.5125
Final evaluation accuray results: 0.8425
